# First Ingestion Operation

Use this github repo to update to lastest pdf file reading engine:

In [ ]:
%pip install git+https://github.com/FzliangFrank/pdf_bank_statement.git

Retrived data should looks like this:

In [1]:
import duckdb

DB_PATH = '/Users/frankliang/Dropbox/Apps/DuckDb/personal.db'
with duckdb.connect(DB_PATH) as con:
    print(con.sql("SELECT * FROM my.bank_statement").limit(3))

┌───────┬─────────────────────┬──────────────────┬───┬───────────────┬─────────────┬───────────┬──────────────────────┐
│  Id   │        Date         │   Description    │ … │ Money Out (£) │ Balance (£) │ Category  │       src_file       │
│ int32 │    timestamp_ns     │     varchar      │   │    double     │   double    │  varchar  │       varchar        │
├───────┼─────────────────────┼──────────────────┼───┼───────────────┼─────────────┼───────────┼──────────────────────┤
│     1 │ 2023-10-02 00:00:00 │ STGCOACH/CTYLINK │ … │           2.0 │      5770.0 │ Transport │ /Users/frankliang/…  │
│     2 │ 2023-10-02 00:00:00 │ PRET A MANGER    │ … │           5.0 │      5764.0 │ Cafe      │ /Users/frankliang/…  │
│     3 │ 2023-10-02 00:00:00 │ STGCOACH/CTYLINK │ … │           2.0 │      5762.0 │ Transport │ /Users/frankliang/…  │
├───────┴─────────────────────┴──────────────────┴───┴───────────────┴─────────────┴───────────┴──────────────────────┤
│ 3 rows                                

### Incremental Update

First check if there are any new files added to my directory since last time. 
So I think best way is to write a table of list file that has been ingested everytime we do something without analysing the whole data table. 

So everytime we scan the file, we just compare what has been ingested versus what has not being ingested. And the only the "set difference" gets ingested. 

### Ad Hoc Analytic
Best practice is to just know what you want to know from this 

In [5]:
con = duckdb.connect(DB_PATH, read_only=True)
bs = con.sql('SELECT * FROM my.bank_statement')
bs.aggregate('''
             min(Date)::datetime::date as "Earliest Transaction", 
             max(Date)::datetime::date as "Latest Transaction",
             sum("Money out (£)") FILTER (WHERE Category != 'Rent & Essential')
                / datediff('month', min(Date), max(Date))
                as "Avg £ Out Per Month (Net Rent)",
             sum("Money in (£)")
                / datediff('month', min(Date), max(Date))
                as "Avg £ In",
             ''') # now the problem here is aggregate method in duckdb is different to ibis

┌──────────────────────┬────────────────────┬────────────────────────────────┬──────────┐
│ Earliest Transaction │ Latest Transaction │ Avg £ Out Per Month (Net Rent) │ Avg £ In │
│         date         │        date        │             double             │  double  │
├──────────────────────┼────────────────────┼────────────────────────────────┼──────────┤
│ 2023-01-03           │ 2023-11-30         │                         2312.7 │   2786.7 │
└──────────────────────┴────────────────────┴────────────────────────────────┴──────────┘

In [87]:
since_date = con.sql("select time_bucket(interval '2 months', today() - interval 1 month) as since_d")

print(since_date)
(
    con.sql("select * from my.bank_statement")
    .aggregate('''
               Category, 
               sum("Money Out (£)") Total,
               round(sum("Money Out (£)") 
                / datediff('Month',min(Date), max(Date)), 2)
                as "Per Month",
               sum("Money Out (£)")
                FILTER(WHERE Date::datetime::date >= (select since_d from since_date))
                as "Per Month Since"
               ''',
               "Category"
               )
)

┌────────────┐
│  since_d   │
│    date    │
├────────────┤
│ 2023-11-01 │
└────────────┘



┌──────────────────┬─────────┬───────────┬─────────────────┐
│     Category     │  Total  │ Per Month │ Per Month Since │
│     varchar      │ double  │  double   │     double      │
├──────────────────┼─────────┼───────────┼─────────────────┤
│ Cloth & Shopping │  2453.0 │    272.56 │            51.0 │
│ Rent & Essential │  5156.0 │     515.6 │          1177.0 │
│ Transport        │   593.0 │      59.3 │            41.0 │
│ Subscri & Apple  │  4306.0 │     430.6 │            81.0 │
│ Food & Grocery   │  2533.0 │     253.3 │           221.0 │
│ Cafe             │   917.0 │    101.89 │            38.0 │
│ Other            │ 12325.0 │    1232.5 │          2302.0 │
└──────────────────┴─────────┴───────────┴─────────────────┘

## Setup

### Bulk read All Files
Read all pdf files stored in `BANK_STATMENT` on a drop box and re-analyse them

In [2]:
# bulk read (only do this)
import os
import pdf_bank_statement.analyser as analyser
import pdf_bank_statement.reader as reader

BANK_STATEMENT = "/Users/frankliang/Dropbox/Apps/Bank-Statement-Reader/BANK_STATEMENT"
all_files = os.listdir(BANK_STATEMENT)

all_statement=[*map(lambda x: BANK_STATEMENT + '/' + x, all_files)]


df = reader.read(all_statement)
analysed=analyser.analyse(df) # categorise mainly

Read... Total 1 tables populated.
Read... Total 2 tables populated.
Read... Total 3 tables populated.
Read... Total 4 tables populated.
Read... Total 5 tables populated.
Read... Total 6 tables populated.
Read... Total 7 tables populated.
Read... Total 8 tables populated.
Read... Total 9 tables populated.
Read... Total 10 tables populated.
Read... Total 11 tables populated.
cleaning complete.


### First Time Bluk Create a Table

Never run this script after everything has been set up. Because we will 
use reference number id to store which transaction are marked as varible, 
which one are none variable.


**Concerns For Multiple Bank Account**

Before we do so we must consider for point where we may actually open different bank statement or we end up unable to tell which account is which...

So it maybe worth creating a foregin key on a column. Add additional table here [Reference](https://www.devart.com/dbforge/sql/studio/add-column-to-table-sql-server.html#:~:text=The%20basic%20syntax%20of%20the,to%20add%20a%20new%20column.) 

In [3]:
import duckdb
im_con = duckdb.connect()
analysed_data = analysed.data
analysed_data.sample(3)
duckdb.from_df(analysed_data.iloc[:,0:8].sample(3))

┌─────────────────────┬────────────────────┬─────────┬───┬─────────────┬────────────────┬──────────────────────┐
│        Date         │    Description     │  Type   │ … │ Balance (£) │    Category    │       src_file       │
│    timestamp_ns     │      varchar       │ varchar │   │   double    │    varchar     │       varchar        │
├─────────────────────┼────────────────────┼─────────┼───┼─────────────┼────────────────┼──────────────────────┤
│ 2023-01-19 00:00:00 │ CO-OP GROUP 070527 │ DEB     │ … │      2597.0 │ Food & Grocery │ /Users/frankliang/…  │
│ 2023-02-28 00:00:00 │ TESCO STORES 2487  │ DEB     │ … │      4642.0 │ Food & Grocery │ /Users/frankliang/…  │
│ 2023-10-02 00:00:00 │ Zettle_*Sundays    │ DEB     │ … │      5663.0 │ Other          │ /Users/frankliang/…  │
├─────────────────────┴────────────────────┴─────────┴───┴─────────────┴────────────────┴──────────────────────┤
│ 3 rows                                                                                   8 col

In [ ]:
import duckdb

DB_PATH = '/Users/frankliang/Dropbox/Apps/DuckDb/personal.db'
with duckdb.connect(DB_PATH) as con:
    try:
        con.sql('''
                DROP TABLE my.bank_statement;
                DROP SEQUENCE seq_bank_statement;
                ''')
        print("Table droped")
    except:
       pass
    finally:
        print("Creating new table")
        con.sql('''
                CREATE SCHEMA IF NOT EXISTS my;
                CREATE SEQUENCE IF NOT EXISTS seq_bank_statement START 1;
                CREATE TABLE my.bank_statement (
                    Id INTEGER PRIMARY KEY,
                    Date timestamp_ns,
                    Description VARCHAR,
                    Type VARCHAR(3),
                    "Money In (£)" DOUBLE,
                    "Money Out (£)" DOUBLE,
                    "Balance (£)" DOUBLE,
                    "Category" VARCHAR(32),
                    src_file VARCHAR(100)
                );
                INSERT INTO my.bank_statement
                SELECT nextval('seq_bank_statement'),* 
                FROM analysed_data
        ''')
        print("Sampled Data Looks Like this")
        print(con.sql("Select * from my.bank_statement order by random() limit 3"))

    